In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [44]:
import pandas as pd
import numpy as np
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm
from sklearn import mixture
from sklearn.mixture import BayesianGaussianMixture
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV

In [45]:
train = pd.read_csv('C:\\Users\\mjafarlou1\\Desktop\\ELL\\train.csv')
test = pd.read_csv('C:\\Users\\mjafarlou1\\Desktop\\ELL\\test.csv')
submit = pd.read_csv('C:\\Users\\mjafarlou1\\Desktop\\ELL\\sample_submission.csv')

In [46]:
train['full_text'] = train['full_text'].apply(lambda x: x.lower())
test['full_text'] = test['full_text'].apply(lambda x: x.lower())

story = []
for doc in train['full_text']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [47]:
model = gensim.models.Word2Vec(
    vector_size=200,
    window=5,
    min_count=2
)

model.build_vocab(story)

model.train(story, total_examples=model.corpus_count, epochs=5)

(5559269, 8109125)

In [48]:
len(model.wv.index_to_key)

10676

In [49]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [50]:
from tqdm import tqdm
X = []
for doc in tqdm(train['full_text'].values):
    w2v=document_vector(doc)
    X.append(w2v)

100%|██████████████████████████████████████████████████████████████████████████████| 3911/3911 [01:57<00:00, 33.23it/s]


In [51]:
from tqdm import tqdm
X1 = []
for doc in tqdm(test['full_text'].values):
    X1.append(document_vector(doc))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 33.99it/s]


In [52]:
X = np.array(X)
X1 = np.array(X1)

In [53]:
columns = train.columns.to_list()[2::]
metric1 = []

for col in tqdm(columns):
    
    y = ((train[col]*2)-2).values
    y = y.astype('int64')
    
    
    
    X_resampled, y_resampled = RandomOverSampler().fit_resample(X, y)
    
    #X_train,X_test,y_train,y_test = train_test_split(X_resampled, y_resampled,test_size=0.2,random_state=1, stratify=y_resampled)
    
    dtree_model = RandomForestRegressor(random_state=1)

    dtree_model.fit(X_resampled, y_resampled)
    
    y_pred = dtree_model.predict(X_resampled)
    
    metric1.append(np.sqrt(mean_squared_error(y_resampled, y_pred)))
    
    test[col] = (np.clip((dtree_model.predict(X1)+2)/2, 1, 5)).astype('float64')
    #print(grid.best_params_)
                  
print('metric:', '\t', np.mean(metric1))    
test

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [10:50<00:00, 108.38s/it]

metric: 	 0.18098597958111964


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,when a person has no experience on a job their...,3.255,3.060,3.065,3.010,2.570,3.10
1,000BAD50D026,do you think students would benefit from being...,3.185,2.805,2.985,2.895,2.915,2.99
2,00367BB2546B,"thomas jefferson once states that ""it is wonde...",3.455,3.170,3.535,3.375,3.265,3.32


In [54]:
test.drop(['full_text'], axis=1, inplace=True)
test

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.255,3.060,3.065,3.010,2.570,3.10
1,000BAD50D026,3.185,2.805,2.985,2.895,2.915,2.99
2,00367BB2546B,3.455,3.170,3.535,3.375,3.265,3.32


In [27]:
X

array([[-0.45171875, -0.06575018, -0.0152784 , ..., -0.19296202,
        -0.44129378, -0.3610303 ],
       [-0.14648592,  0.08106826,  0.05594174, ..., -0.20188716,
        -0.36851802, -0.45461655],
       [-0.2573076 ,  0.12816374, -0.12199082, ..., -0.17975673,
        -0.44584498, -0.18815804],
       ...,
       [-0.17771392, -0.03886845,  0.02306715, ..., -0.18903524,
        -0.41661936, -0.3787562 ],
       [-0.02274419, -0.03295993,  0.00403278, ..., -0.23277405,
        -0.35421982, -0.42752478],
       [-0.05372131,  0.17252055,  0.02505984, ..., -0.18969907,
        -0.5608798 , -0.52511215]], dtype=float32)

In [28]:
############################
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

chain = MultiOutputRegressor(SVR())
chain.fit(X, y)
print(chain.score(X,y))

ValueError: y must have at least two dimensions for multi-output regression but has only one.

In [55]:
predictions = chain.predict(X_test_tfIdf)
submission['vocabulary'] = predictions[:,2]
submission['vocabulary'] 


NameError: name 'X_test_tfIdf' is not defined